In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

fairuzazaria_rtm_stuck_prediction_dataframes_path = kagglehub.dataset_download('fairuzazaria/rtm-stuck-prediction-dataframes')

print('Data source import complete.')


# **DATA TRANSFORMATION**

In [ ]:
step_in   = 60
step_out  = 60
scaling   = False
well_name = "well_a"
scale_type = "no_scale"
use_additionals = True

if scaling:
    scale_type = "minmax"

In [ ]:
#-- initialize variables
line_plot_conf = {
    "grid"     : True,
    "sharex"   : True,
    "figsize"  : (12, 16),
    "subplots" : True,
}

pie_plot_conf = {
    "autopct" : "%1.1f%%",
    "figsize" : (5, 5),
}

# **1. PREPARATION**

In [ ]:
!pip install fastparquet
!pip install pandas==2.2.3

## **1.1. IMPORT LIBRARIES**

In [ ]:
import os
import glob
import h5py
import pyarrow
import datetime
import fastparquet

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from scipy.stats import kurtosis, skew, linregress

In [ ]:
from tqdm.notebook import tqdm
from multiprocessing import Lock

tqdm.set_lock(Lock())

## **1.2. PREPARE FUNCTIONS**

In [ ]:
#-- func: get all dataset directories
def get_dataset_directories(pattern: str = 'txt', base: str = '/kaggle/input') -> tuple:
    working_dir = glob.glob(os.path.join(base, '*'), recursive=True)
    dataset_dir = tuple(
        tuple(
            filename for filename in glob.iglob(os.path.join(dir, '**', f'*{pattern}'), recursive=True)
        )
        for dir in working_dir
    )

    return (dataset_dir, working_dir)

In [ ]:
#-- func: check missing timestamps in dataset
def get_missing_timestamps(dataframe: pd.DataFrame, date_column: str, freq: str = '10s') -> tuple:
    missing = pd.date_range(
        start = dataframe[date_column].min(),
        end   = dataframe[date_column].max(),
        freq  = freq
    )

    return tuple(missing.difference(dataframe[date_column]))

In [ ]:
#-- func: apply sliding window
def split_sequences(sequences, n_steps_in, n_steps_out):
  X, y = list(), list()
  for i in tqdm(range(len(sequences))):
    end_ix = i + n_steps_in
    out_end_ix = end_ix + n_steps_out

    if out_end_ix > len(sequences):
      break

    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[out_end_ix - 1, -1]
    X.append(seq_x)
    y.append(seq_y)

  return np.array(X), np.array(y)

In [ ]:
def vectorized_statistical_features(X):
    mins   = np.min(X, axis=2, keepdims=True)
    maxs   = np.max(X, axis=2, keepdims=True)
    ranges = maxs - mins

    X_2d = X.reshape(-1, X.shape[2])
    mask = ~np.all(X_2d == X_2d[:, [0]], axis=1)

    skew_vals = np.zeros(X_2d.shape[0])
    kurt_vals = np.zeros(X_2d.shape[0])
    skew_vals[mask] = skew(X_2d[mask], axis=1)
    kurt_vals[mask] = kurtosis(X_2d[mask], axis=1)

    diffs = np.diff(X, axis=2)
    stats = np.concatenate([
        mins, maxs,
        np.mean(X, axis=2, keepdims=True),
        np.std(X, axis=2, keepdims=True),
        np.median(X, axis=2, keepdims=True),
        skew_vals.reshape(X.shape[0], X.shape[1], 1),
        kurt_vals.reshape(X.shape[0], X.shape[1], 1),
        np.mean(diffs, axis=2, keepdims=True),
        np.std(diffs, axis=2, keepdims=True),
        np.min(diffs, axis=2, keepdims=True),
        np.max(diffs, axis=2, keepdims=True),
        ranges
    ], axis=2)

    return stats

In [ ]:
#-- func: remove placeholder value and make it nan
def set_placeholder_value_to_nan(dataframe: pd.DataFrame, features: list, placeholder: str = '-') -> None:
    for feature in tqdm(features):
        dataframe[feature] = dataframe[feature].apply(lambda value: np.nan if value == placeholder else value)

In [ ]:
#-- func: scale every features
def feature_wise_minmax(X):
    num_samples, timesteps, num_features = X.shape
    X_scaled = np.zeros_like(X)

    scalers = []
    for i in range(num_features):
        scaler  = MinMaxScaler()
        feature = X[:, :, i].reshape(-1, 1)
        X_scaled[:, :, i] = scaler.fit_transform(feature).reshape(num_samples, timesteps)

        scalers.append(scaler)

    return X_scaled, scalers

In [ ]:
#-- func: transform every features
def feature_wise_minmax_transform(X, scalers):
    num_samples, timesteps, num_features = X.shape
    X_scaled = np.zeros_like(X)

    for i in range(num_features):
        feature = X[:, :, i].reshape(-1, 1)
        X_scaled[:, :, i] = scalers[i].transform(feature).reshape(num_samples, timesteps)

    return X_scaled

# **2. PREPROCESS DATA**

## **2.1. READ DATASET**

In [ ]:
#-- read dataset
ph = '/kaggle/input/rtm-stuck-prediction-dataframes/well_a.parquet'
df = pq.read_table(ph).to_pandas()

print(f'fetched well a data with shape {df.shape}')

In [ ]:
#-- define unused feature
features = [
    "Date-Time", "LogDepth",
    "BitDepth", "BlockPos",
    "Hkld", "MudFlowIn",
    "ROPi", "RPM",
    "Torque", "SpPress",
    "WOB", "Stuck"
]

df = df[features]

In [ ]:
#-- convert string datetime to datetime
df['Date-Time'] = pd.to_datetime(df['Date-Time'])
df = df.sort_values(by = ['Date-Time'])
df = df.reset_index(drop=True)

df.head()

In [ ]:
#-- check placeholder value counts
placeholder = '-'
placeholder_df  = {feature: np.sum(df[feature] == placeholder) for feature in features}
df_placeholder = pd.DataFrame.from_dict([placeholder_df])
df_placeholder.head()

In [ ]:
#-- dataset info
print(f'well a shape      : {df.shape}')
print(f"well a labels     : {list(df['Stuck'].value_counts())}")
print(f'well a missing    : {len(get_missing_timestamps(df, df.columns[0]))}')
print(f'well a uplicates  : {df["Date-Time"].duplicated().sum()}')
print(f'well a nan values : \n\n{df.isna().sum()}\n')

In [ ]:
#-- replace placeholder data
set_placeholder_value_to_nan(df, features)
df[df.columns[1:-1]] = df[df.columns[1:-1]].astype(float)

In [ ]:
#-- dataset info
print(f'well a shape      : {df.shape}')
print(f"well a labels     : {list(df['Stuck'].value_counts())}")
print(f'well a missing    : {len(get_missing_timestamps(df, df.columns[0]))}')
print(f'well a uplicates  : {df["Date-Time"].duplicated().sum()}')
print(f'well a nan values : \n\n{df.isna().sum()}\n')

In [ ]:
#-- fill missing timestamp
df_ms = get_missing_timestamps(df, df.columns[0])
df_m = pd.DataFrame({"Date-Time": df_ms})
df   = pd.concat([df, df_m]).drop_duplicates(subset=["Date-Time"]).sort_values("Date-Time")

df[df.columns[1:-1]] = df[df.columns[1:-1]].interpolate()
df['Stuck'] = df['Stuck'].ffill()

In [ ]:
#-- dataset info
print(f'well a shape      : {df.shape}')
print(f"well a labels     : {list(df['Stuck'].value_counts())}")
print(f'well a missing    : {len(get_missing_timestamps(df, df.columns[0]))}')
print(f'well a uplicates  : {df["Date-Time"].duplicated().sum()}')
print(f'well a nan values : \n\n{df.isna().sum()}\n')
print(f'well a negatives  : \n\n{(df[df.columns[1:-1]] < 0).sum()}\n')

## **2.2. STANDARDIZE TIMESTAMPS**

In [ ]:
#-- standardize timestamps
df_resampled = df[df.columns[:-1]].set_index("Date-Time").resample("5s").ffill()
df_resampled["Stuck"] = df.set_index("Date-Time")['Stuck'].resample("5s").ffill()
df_resampled = df_resampled.reset_index()

df = df_resampled
del df_resampled

In [ ]:
#-- dataset info
print(f'well a shape      : {df.shape}')
print(f"well a labels     : {list(df['Stuck'].value_counts())}")
print(f'well a missing    : {len(get_missing_timestamps(df, df.columns[0], freq = "5s"))}')
print(f'well a uplicates  : {df["Date-Time"].duplicated().sum()}')
print(f'well a nan values : \n\n{df.isna().sum()}\n')
print(f'well a negatives  : \n\n{(df[df.columns[1:-1]] < 0).sum()}\n')

## **2.3. DATA ANALYSIS**

In [ ]:
#-- cut ST data
df = df[(df["Date-Time"] <= (datetime.datetime(2023, 7, 1, 0, 0)))]

In [ ]:
#-- check each max data:
print(f'well a maxs : \n{df[df.columns[1:-1]].max()}\n')
print(f'well a mins : \n{df[df.columns[1:-1]].min()}\n')

In [ ]:
#-- check unusual MudFlowIn values
df[df["MudFlowIn"] > 2000]

In [ ]:
df.iloc[598660:598670]

In [ ]:
df[df["ROPi"] > 600]

In [ ]:
df[df["RPM"] > 300]

In [ ]:
df[700070:700080]

In [ ]:
#-- check error ropi values
df[(df["ROPi"] > 0) & (df["RPM"] == 0) & (df["Torque"] == 0)]

## **2.4. HANDLE OUTLIERS**

In [ ]:
#-- replace ROPi outliers
df["ROPi"] = df.apply(lambda row: 0 if (
    row["ROPi"] > 0 and
    row["RPM"] == 0 and
    row["Torque"] == 0
) else row["ROPi"], axis=1)

In [ ]:
#-- replace MudFlowIn outliers
df["MudFlowIn"] = df["MudFlowIn"].apply(lambda value: np.nan if (value > 2000) else value)
df["MudFlowIn"] = df["MudFlowIn"].interpolate().ffill()

In [ ]:
#-- replace MudFlowIn outliers
df["RPM"] = df["RPM"].apply(lambda value: np.nan if (value > 300) else value)
df["RPM"] = df["RPM"].interpolate().ffill()

In [ ]:
#-- replace negative values with nan
for column in tqdm(df.columns[1:-1]):
    df[column] = df[column].apply(lambda value: np.nan if value < 0 else value)

In [ ]:
#-- check each max data:
print(f'well a maxs : \n{df[df.columns[1:-1]].max()}\n')
print(f'well a mins : \n{df[df.columns[1:-1]].min()}\n')

## **2.5. FILL NAN VALUES**

In [ ]:
#-- interpolate nan values
df["BitDepth"] = df["BitDepth"].interpolate()
df["BlockPos"] = df["BlockPos"].interpolate()

#-- clip negatives
df[df.columns[1:-1]] = df[df.columns[1:-1]].clip(lower=0)

In [ ]:
#-- dataset info
print(f'well a shape      : {df.shape}')
print(f"well a labels     : {list(df['Stuck'].value_counts())}")
print(f'well a missing    : {len(get_missing_timestamps(df, df.columns[0], freq = "5s"))}')
print(f'well a uplicates  : {df["Date-Time"].duplicated().sum()}')
print(f'well a nan values : \n\n{df.isna().sum()}\n')
print(f'well a negatives  : \n\n{(df[df.columns[1:-1]] < 0).sum()}\n')

## **2.6. FEATURE EXTRACTION**

In [ ]:
#-- get differences
df["RpmDiff"]      = df["RPM"].diff().fillna(0)
df["HkldDiff"]     = df["Hkld"].diff().fillna(0)
df["TorqueDiff"]   = df["Torque"].diff().fillna(0)
df["BitDepthDiff"] = df["BitDepth"].diff().fillna(0)
df["LogDepthDiff"] = df["LogDepth"].diff().fillna(0)

In [ ]:
df["BitDepthRollMin"] = df["BitDepth"].rolling(180).min().fillna(0)
df["BitDepthRollMax"] = df["BitDepth"].rolling(180).max().fillna(0)

df["BitDepthRollMinDiff"] = df["BitDepthRollMin"].diff().fillna(0)
df["BitDepthRollMaxDiff"] = df["BitDepthRollMax"].diff().fillna(0)

df["HkldRollMin"] = df["Hkld"].rolling(180).min().fillna(0)
df["HkldRollMax"] = df["Hkld"].rolling(180).max().fillna(0)

In [ ]:
#-- check ROPi
miss_rop = len(df[(df["ROPi"] == 0) & (df["RPM"] > 0) & (df["Torque"] > 0) & (df["BitDepthDiff"] > 0) & (df["LogDepthDiff"] > 0)])
print(f"miscalculated ROPi: {miss_rop}")

In [ ]:
#-- adjust ROPi
df["ROPi"] = df.apply(lambda row: 1e-6 if (
    row["ROPi"] == 0 and
    row["RPM"] > 0 and
    row["Torque"] > 0 and
    row["BitDepthDiff"] > 0 and
    row["LogDepthDiff"] > 0
) else row["ROPi"], axis=1)

In [ ]:
#-- check ROPi
miss_rop = len(df[(df["ROPi"] == 0) & (df["RPM"] > 0) & (df["Torque"] > 0) & (df["BitDepthDiff"] > 0) & (df["LogDepthDiff"] > 0)])
print(f"miscalculated ROPi: {miss_rop}")

In [ ]:
#-- check bit location
df["BitInsideWell"] = df.apply(lambda row: 1 if (row["BitDepth"] != 0) else 0, axis=1)

In [ ]:
#-- check if unstable shell exist
df["ShellOnBitString"] = df.apply(lambda row: 1 if (
    (0 <= row["BitDepthRollMax"] - row["BitDepthRollMin"] <= 1) and
    row["RPM"] > 0 and
    row["Torque"] > 0
) else 0, axis=1)

In [ ]:
#-- check for bit drill
df["HIghForceNoRotation"] = df.apply(lambda row: 1 if (
    row["RPM"] == 0 and
    row["Torque"] > 0
) else 0, axis=1)

In [ ]:
#-- calculate drag
df["Drag"] = df.apply(lambda row: row["HkldDiff"] if (row["BitDepthDiff"] != 0) else 0, axis=1)
df["Drag"] = df.apply(lambda row: 0 if (row["Drag"] < 0) else row["Drag"], axis=1)

In [ ]:
#-- calculate overpull
df["Overpull"] = df.apply(lambda row: row["Hkld"] - row["HkldRollMin"] if (
    row["BitDepthDiff"] == 0 and
    row["Hkld"] > row["HkldRollMin"]
) else 0, axis=1)

In [ ]:
#-- check pump flow rate
df["PumpFlow"] = df.apply(lambda row: 1 if row["MudFlowIn"] != 0 else 0, axis=1)

In [ ]:
#-- check stall
df["Stall"] = df.apply(lambda row: 1 if (
    row["RPM"] == 0 and
    row["RpmDiff"] < 0 and
    row["TorqueDiff"] > 0
) else 0, axis=1)

In [ ]:
#-- check POOH and RIH
df["POOH"] = df.apply(lambda row: 1 if (
    row["BitDepthRollMinDiff"] < 0 and
    row["ROPi"] == 0 and
    row["WOB"] == 0 and
    row["LogDepthDiff"] == 0
) else 0, axis=1)

df["RIH"] = df.apply(lambda row: 1 if (
    row["BitDepthRollMaxDiff"] > 0 and
    row["ROPi"] == 0 and
    row["WOB"] == 0 and
    row["LogDepthDiff"] == 0
) else 0, axis=1)

In [ ]:
#-- check general activities
df["reaming"] = df.apply(lambda row: 1 if (
    row["RPM"] > 0 and
    row["WOB"] == 0 and
    row["ROPi"] == 0 and
    row["Torque"] > 0 and
    row["SpPress"] >= 0 and
    row["MudFlowIn"] > 0
) else 0, axis=1)

df["drilling"] = df.apply(lambda row: 1 if (
    row["WOB"] > 0 and
    row["RPM"] > 0 and
    row["ROPi"] > 0 and
    row["Torque"] >= 0 and
    row["LogDepthDiff"] > 0
) else 0, axis=1)

df["connection"] = df.apply(lambda row: 1 if (
    row["RPM"] == 0 and
    row["WOB"] == 0 and
    row["ROPi"] == 0 and
    row["LogDepthDiff"] == 0
) else 0, axis=1)

df["others"] = df.apply(lambda row: 1 if (
    row["reaming"] == 0 and
    row["drilling"] == 0 and
    row["connection"] == 0
) else 0, axis=1)

In [ ]:
#-- drop unused columns
df_stuck = df["Stuck"]
columns  = [
    "LogDepth",
    "BitDepth",
    "BlockPos",
    "BitDepthDiff",
    "LogDepthDiff",
    "HkldDiff",
    "RpmDiff",
    "TorqueDiff",
    "BitDepthRollMin",
    "BitDepthRollMax",
    "BitDepthRollMinDiff",
    "BitDepthRollMaxDiff",
    "HkldRollMin",
    "HkldRollMax",
    "Stuck"
]

df = df.drop(columns=columns)
df["Stuck"] = df_stuck

In [ ]:
#-- check activity overlapp
print(df.loc[(df["reaming"] == 1) & (df["drilling"] == 1)].shape)
print(df.loc[(df["reaming"] == 1) & (df["connection"] == 1)].shape)
print(df.loc[(df["connection"] == 1) & (df["drilling"] == 1)].shape)
print(df.loc[(df["connection"] == 1) & (df["reaming"] == 1)].shape)
print(df.loc[(df["drilling"] == 1) & (df["reaming"] == 1)].shape)
print(df.loc[(df["drilling"] == 1) & (df["connection"] == 1)].shape)

print(df.loc[(df["reaming"] == 1) & (df["drilling"] == 1) & (df["connection"] == 1)].shape)
print(df.loc[(df["reaming"] == 0) & (df["drilling"] == 0) & (df["connection"] == 0) & (df["others"] == 0)].shape)

In [ ]:
#-- dataset info
print(f'well a shape      : {df.shape}')
print(f"well a labels     : {list(df['Stuck'].value_counts())}")
print(f'well a missing    : {len(get_missing_timestamps(df, df.columns[0], freq = "5s"))}')
print(f'well a uplicates  : {df["Date-Time"].duplicated().sum()}')
print(f'well a nan values : \n\n{df.isna().sum()}\n')
print(f'well a negatives  : \n\n{(df[df.columns[1:-1]] < 0).sum()}\n')

In [ ]:
# df.set_index("Date-Time").plot(**line_plot_conf)

## **2.7. APPLY SLIDING WINDOW**

In [ ]:
#-- differentiate features
num_features = df.columns[1:]
num_feature_con = [
    'Hkld', 'MudFlowIn',
    'ROPi', 'RPM', 'Torque',
    'SpPress', 'WOB', 'Drag',
    'Overpull', 'Stuck'
]

num_feature_cat = [
    'BitInsideWell', 'ShellOnBitString',
    'HIghForceNoRotation', 'PumpFlow',
    'Stall', 'POOH', 'RIH', 'reaming',
    'drilling', 'connection', 'others', 'Stuck'
]

In [ ]:
#-- apply sliding window
df_X_con = df.set_index("Date-Time")[num_feature_con].astype("float32").values
df_X_cat = df.set_index("Date-Time")[num_feature_cat].astype("float32").values

X_con, y_con = split_sequences(df_X_con, step_in, step_out)
X_cat, y_cat = split_sequences(df_X_cat, step_in, step_out)

In [ ]:
#-- get X shape
print(f'X_con : {X_con.shape}')
print(f'X_cat : {X_cat.shape}')
print(f'y_con : {y_con.shape}')
print(f'y_cat : {y_cat.shape}')

In [ ]:
split_point_con = int((len(df_X_con) - step_in + 1) * 0.8)
split_point_cat = int((len(df_X_cat) - step_in + 1) * 0.8)

In [ ]:
#-- split dataset
X_con_train, X_con_test = X_con[:split_point_con], X_con[split_point_con:]
y_con_train, y_con_test = y_con[:split_point_con], y_con[split_point_con:]
del X_con, y_con

X_cat_train, X_cat_test = X_cat[:split_point_cat], X_cat[split_point_cat:]
y_cat_train, y_cat_test = y_cat[:split_point_cat], y_cat[split_point_cat:]
del X_cat, y_cat

In [ ]:
print(f'continous shape : {X_con_train.shape, X_con_test.shape}')
print(f'static shape    : {X_cat_train.shape, X_cat_test.shape}')

In [ ]:
print(np.unique(y_con_train, return_counts=True))
print(np.unique(y_cat_train, return_counts=True))

In [ ]:
print(np.unique(y_con_test, return_counts=True))
print(np.unique(y_cat_test, return_counts=True))

## **2.8. SCALE DATASET**

In [ ]:
#-- scale dataset
if scaling:
    X_con_train, scalers = feature_wise_minmax(X_con_train)
    X_con_test = feature_wise_minmax_transform(X_con_test, scalers)

del y_cat_train, y_cat_test

In [ ]:
if use_additionals:
    X_train = np.concatenate([X_con_train, X_cat_train], axis=2)
    del X_con_train, X_cat_train

    X_test  = np.concatenate([X_con_test, X_cat_test], axis=2)
    del X_con_test, X_cat_test

    y_train = y_con_train
    y_test  = y_con_test
    del y_con_train, y_con_test
else:
    X_train = X_con_train
    X_test  = X_con_test
    y_train = y_con_train
    y_test  = y_con_test

In [ ]:
#-- get shapes
print(f'X_train : {X_train.shape}')
print(f'X_test  : {X_test.shape}')
print(f'y_train : {y_train.shape}')
print(f'y_test  : {y_test.shape}')

# **3. SAVE DATASET**

In [ ]:
#-- get directories
train_path = os.path.join(os.getcwd(), f"{well_name}_train_adds_normal_{scale_type}_{step_in}{step_out}_0_new.h5")
test_path  = os.path.join(os.getcwd(), f"{well_name}_test_adds_normal_{scale_type}_{step_in}{step_out}_0_new.h5")

train_path, test_path

In [ ]:
#-- save training data
n_samples, timesteps, features = X_train.shape
batch_size = 10000

with h5py.File(train_path, "w") as f:
    dset_X = f.create_dataset(
        "X", shape=(n_samples, timesteps, features),
        dtype="float32", compression="gzip", chunks=(batch_size, timesteps, features)
    )
    dset_y = f.create_dataset(
        "y", shape=(n_samples,),
        dtype="float16", compression="gzip", chunks=(batch_size,)
    )

    for i in range(0, n_samples, batch_size):
        end = min(i + batch_size, n_samples)

        X_batch = X_train[i:end]
        y_batch = y_train[i:end]

        dset_X[i:end] = X_batch
        dset_y[i:end] = y_batch

In [ ]:
#-- save test data
n_samples, timesteps, features = X_test.shape
batch_size = 1000

with h5py.File(test_path, "w") as f:
    dset_X = f.create_dataset(
        "X", shape=(n_samples, timesteps, features),
        dtype="float32", compression="gzip", chunks=(batch_size, timesteps, features)
    )
    dset_y = f.create_dataset(
        "y", shape=(n_samples,),
        dtype="float16", compression="gzip", chunks=(batch_size,)
    )

    for i in range(0, n_samples, batch_size):
        end = min(i + batch_size, n_samples)

        X_batch = X_test[i:end]
        y_batch = y_test[i:end]

        dset_X[i:end] = X_batch
        dset_y[i:end] = y_batch